# Problem-Specific Coordinate Generation for HyperNEAT Substrates

# Part 3: Neuroevolution

In this part, the created substrates are tested on the defined problem.

## Setup

### Imports

In [ ]:
import os
import pickle
from collections import defaultdict

import wandb

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from config import config

import jax
from tensorneat.common import State

from evol_pipeline.brax_env import CustomBraxEnv
from evol_pipeline.custom_pipeline import CustomPipeline
from evol_pipeline.evol_algorithm import create_evol_algorithm

from utils.visualization import visualize_cppn, visualize_nn
from utils.utils import setup_folders_evolution, append_summary_row

The folder structure is checked and setup if necessary. A deep dictionary is defined and will hold the different substrates.

In [ ]:
OUTPUT_DIR = config["experiment"]["output_dir_evolultion"]
setup_folders_evolution(OUTPUT_DIR)

def deep_defaultdict():
    return defaultdict(deep_defaultdict)

### Setup Environment

[Brax environments](https://github.com/google/brax/tree/main/brax/envs) are used for this experiment through the [TensorNEAT wrapper](https://github.com/EMI-Group/tensorneat/tree/main/src/tensorneat/problem/rl).

In [3]:
env_name = config["experiment"]["env_name"]
env_problem = CustomBraxEnv(
    env_name=env_name,
    backend=config["environment"]["backend"],
    brax_args=config["environment"]["brax_args"],
    max_step=config["environment"]["max_step"],
    repeat_times=config["environment"]["repeat_times"],
    obs_normalization=config["environment"]["obs_normalization"],
    sample_episodes=config["environment"]["sample_episodes"],
)
obs_size = env_problem.input_shape[0]
act_size = env_problem.output_shape[0]

/home/andi/anaconda3/envs/jax/lib/python3.10/site-packages/brax/io/mjcf.py:480: UserWarning: Brax System, piplines and environments are not actively being maintained. Please see MJX for a well maintained JAX-based physics engine: https://github.com/google-deepmind/mujoco/tree/main/mjx. For a host of environments that use MJX, see: https://github.com/google-deepmind/mujoco_playground.
  warnings.warn(
2025-11-03 10:33:29.188280: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2025-11-03 10:33:39.948119: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by usin

The substrates which were previously saved to disk are used for the neuroevolution process.

In [4]:
with open(config["experiment"]["substrates_path"], "rb") as f:
    substrates = pickle.load(f)

## Neuroevolution

Finally the substrates are ready to be used for neuroevolution with HyperNEAT. Weights and Biases (wandb) is used for logging and the CPPNs with highest fitness and their phenotypes are visualized. Coordinates are logged for further analysis.


In [ ]:
for data_label, data_dict in substrates.items():
    for method_label, method_dict in data_dict.items():
        for feature_dims_label, feature_dims_dict in method_dict.items():

            active_substrate = substrates[data_label][method_label][feature_dims_label]["substrate"]
            evol_algorithm = create_evol_algorithm(substrate=active_substrate)

            run_id = f"{data_label}_{method_label}_{substrate_dimensions}d"

            initial_cppn_layers = config["algorithm"]["genome"]["cppn_init_hidden_layers"](active_substrate.query_coors.shape[1])
            print("Intial CPPN Layers:", initial_cppn_layers)
            substrate_dimensions = int(active_substrate.query_coors.shape[1]/2)

            wanbd_name = run_id
            wandb_tags = [config["substrate"]["hidden_layer_type"], env_name, data_label, method_label, f"{config['substrate']['hidden_depth']}_hl", f"{config['algorithm']['neat']['pop_size']}pop", f"{config['environment']['backend']}"]

            wandb.init(
                name=wanbd_name,
                project="substrate_configuration",
                tags=wandb_tags,
                config=config  
            )

            wandb.config.update(
                {
                    "substrate": {
                        "obs_size": obs_size,
                        "act_size": act_size,
                        "num_queries": active_substrate.query_coors.shape[0],
                        "query_dim": active_substrate.query_coors.shape[1],
                        },
                    "algorithm": {
                        "neat": {
                            "num_inputs": evol_algorithm.num_inputs,
                            },
                        "genome": {
                            "cppn_init_hidden_layers": initial_cppn_layers,
                            },
                        },
                },
            )
            
            genome_save_dir = os.path.join(OUTPUT_DIR, "genome", run_id)
            os.makedirs(genome_save_dir, exist_ok=True)

            pipeline = CustomPipeline(
                algorithm=evol_algorithm,
                problem=env_problem,
                seed=config["experiment"]["seed"],
                generation_limit=config["pipeline"]["generation_limit"],
                fitness_target=config["pipeline"]["fitness_target"],
                is_save=True,
                save_dir=genome_save_dir,
            )

            init_state = pipeline.setup()
            train_state, best_genome = pipeline.auto_run(state=init_state)

            print(f"\nTraining finished. Best fitness achieved: {pipeline.best_fitness}")

            wandb.finish()

            append_summary_row(
                path=f"{OUTPUT_DIR}/summary_results.csv",
                row={
                    "dimensionality": int(active_substrate.query_coors.shape[1] / 2),
                    "sampling": data_label,
                    "method": method_label,
                    "max_fitness": float(pipeline.best_fitness),
                },
            )

            # A fresh state is used for the display with the show method
            showing_state = pipeline.setup()

            # Built-in show method to produce and save a video of the agent
            pipeline.show(
                state=showing_state,
                best=best_genome,
                output_type="mp4",
                save_path=f"{OUTPUT_DIR}/video/agent_{data_label}_{method_label}_{feature_dims_label}.mp4",
            )

            # All coordinates are needed for visualization
            input_coors = substrates[data_label][method_label][feature_dims_label]["input_coors"]
            hidden_coors = substrates[data_label][method_label][feature_dims_label]["hidden_coors"]
            output_coors = substrates[data_label][method_label][feature_dims_label]["output_coors"]

            # Visualizes the CPPN
            visualize_cppn(
                pipeline=pipeline, 
                state=train_state, 
                save_path=f"{OUTPUT_DIR}/topology/cppn_{data_label}_{method_label}_{feature_dims_label}.svg"
            )
            
            # Visualizes a representation of the neural network in 2D space
            visualize_nn(
                pipeline=pipeline, 
                state=train_state, 
                save_path=f"{OUTPUT_DIR}/topology/nn_{data_label}_{method_label}_{feature_dims_label}.svg", 
                substrate=active_substrate, 
                input_coors=input_coors, 
                hidden_coors=hidden_coors, 
                output_coors=output_coors, 
                hidden_depth=config["substrate"]["hidden_depth"], 
                max_weight=config["algorithm"]["hyperneat"]["max_weight"], 
            )

Intial CPPN Layers: [2]


wandb: Currently logged in as: wirkelzirkel (wirkelzirkel-iu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


save to output/ant/genome/manual_simple_1fd
initializing
initializing finished
start compile
compile finished, cost time: 30.312680s
Generation: 1, Cost time: 3345.22ms
 	fitness: valid cnt: 300, max: 51.2546, min: -3955.9634, mean: -1384.1736, std: 1297.4559

	node counts: max: 8, min: 6, mean: 7.25
 	conn counts: max: 12, min: 5, mean: 10.41
 	species: 10, [24, 62, 23, 36, 10, 1, 1, 27, 1, 115]

Generation: 2, Cost time: 3296.62ms
 	fitness: valid cnt: 300, max: 51.4607, min: -3955.0356, mean: -515.2519, std: 1077.9319

	node counts: max: 9, min: 6, mean: 7.55
 	conn counts: max: 14, min: 6, mean: 11.08
 	species: 10, [36, 43, 35, 23, 30, 29, 10, 1, 9, 84]

Generation: 3, Cost time: 3277.15ms
 	fitness: valid cnt: 300, max: 52.0317, min: -3950.1392, mean: -166.3135, std: 721.0425

	node counts: max: 10, min: 6, mean: 7.82
 	conn counts: max: 15, min: 5, mean: 11.55
 	species: 10, [22, 1, 42, 24, 31, 25, 11, 23, 9, 112]

Generation: 4, Cost time: 3291.17ms
 	fitness: valid cnt: 300, m

compute_ms,▆▄▃▄▄▅▆▄▅▅▅▄▃▄▆▅▅▄▄▅▄▆▅▃▃██▇███▃▅▁▁▁▁▂▂▂
compute_ms_pop,▆▄▃▄▄▅▅▄▅▄▅▄▃▄▆▄▅▄▄▅▄▆▅▃▃██▇▆▇███▅▁▂▁▂▂▂
fitness_max,▁▁▄▅▆▅▅▅▆▇▅▆▆▅▆▇▆▇▇▆▆▅▅▅▆▅▆▅▆▇▆▇▆▅▆▇▆▇█▆
fitness_mean,▁▆▇████▇██▇██▇▇██▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▆▇
fitness_min,▁▁▁▁▁▃▁██▅▁▇█▁▃██▁▁▄▁▄▃▁▁▁▁▁▁█▁▂▁█▁▁▁▇▁▁
fitness_std,█▆▃▁▂▁▁▃▂▁▃▃▂▂▃▃▂▃▄▄▂▄▃▄▄▅▅▄▄▄▄▄▄▂▄▃▄▂▅▄
generation,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
num_species,███████████████████████▁█████▁██████████
compute_ms,3258.22425
compute_ms_pop,10.86075
fitness_max,125.85429


initializing
initializing finished


...

mp4 saved to:  output/ant/video/agent_manual_mapping_9fd.mp4
Visualizing CPPN. Saving to output/ant/topology/cppn_manual_mapping_9fd.svg.
Manually reconstructing the phenotype. A visual layout will be generated.
Substrate has 1792 potential connections.
Visualizing 1623 connections. Excluded loops. Weight threshold: 0.005
Visualization saved to: output/ant/topology/nn_manual_mapping_9fd.svg
Successfully saved coordinates to: output/ant/coordinates/manual_mapping_9fd_io.csv
Intial CPPN Layers: [2]


save to output/ant/genome/trained_ica_1fd
initializing
initializing finished
start compile


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x78716872c8e0>> (for post_run_cell), with arguments args (<ExecutionResult object at 787151837d00, execution_count=5 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 787151837f10, raw_cell="for data_label, data_dict in substrates.items():
 .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/andi/Dokumente/Bachelorarbeit/dim_tuning/neuroevolution.ipynb#X56sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
eval_output_dir = f"{OUTPUT_DIR}/eval"
os.makedirs(eval_output_dir, exist_ok=True)

# Load CSV file and set categories
df = pd.read_csv("output/ant/summary_results.csv")
df['method'] = df['method'].astype('category')
df['sampling'] = df['sampling'].astype('category')
df['dimensionality'] = df['dimensionality'].astype('category')

# Main Effect: Method
plt.figure(figsize=(8, 5))
sns.barplot(x='method', y='max_fitness', data=df, errorbar='sd')
plt.title('Main Effect: Reduction Method')
plt.ylabel('Fitness Avg')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(eval_output_dir, "main_effect_method.png"))
plt.close()

# Main Effect: Sampling
plt.figure(figsize=(6, 5))
sns.barplot(x='sampling', y='max_fitness', data=df, errorbar='sd')
plt.title('Main Effect: Data Sampling')
plt.ylabel('Fitness Avg')
plt.tight_layout()
plt.savefig(os.path.join(eval_output_dir, "main_effect_sampling.png"))
plt.close()

# Main Effect: Dimensionality
plt.figure(figsize=(8, 5))
sns.barplot(x='dimensionality', y='max_fitness', data=df, errorbar='sd')
plt.title('Main Effect: Dimensionality')
plt.ylabel('Fitness Avg')
plt.tight_layout()
plt.savefig(os.path.join(eval_output_dir, "main_effect_dimensionality.png"))
plt.close()

# Interaction: Method × Sampling
plt.figure(figsize=(10, 6))
sns.pointplot(x='method', y='max_fitness', hue='sampling', data=df, errorbar='sd', dodge=True)
plt.title('Interaction: Reduction Method × Data Sampling')
plt.ylabel('Fitness Avg')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(eval_output_dir, "interaction_method_sampling.png"))
plt.close()

# Interaction: Method × Dimensionality
plt.figure(figsize=(10, 6))
sns.pointplot(x='dimensionality', y='max_fitness', hue='method', data=df, errorbar='sd', dodge=True)
plt.title('Interaction Effect: Method × Dimensionality')
plt.ylabel('Fitness Avg')
plt.tight_layout()
plt.savefig(os.path.join(eval_output_dir, "interaction_method_dimensionality.png"))
plt.close()

# Interaction: Sampling × Dimensionality
plt.figure(figsize=(10, 6))
sns.pointplot(x='dimensionality', y='max_fitness', hue='sampling', data=df, errorbar='sd', dodge=True)
plt.title('Interaction Effect: Sampling × Dimensionality')
plt.ylabel('Fitness Avg')
plt.tight_layout()
plt.savefig(os.path.join(eval_output_dir, "interaction_sampling_dimensionality.png"))
plt.close()